In [1]:
from vgg16 import vgg16
import tensorflow as tf
import numpy as np
import time   
from tqdm import tqdm
from scipy.misc import imread, imresize
from imagenet_classes import class_names
from PIL import Image
from resizeimage import resizeimage
import glob
import os
import sys


In [4]:
def predict(X):
    old_stdout = sys.stdout
    sys.stdout = open(os.devnull, 'w')
    results = []
    sess = tf.Session()
    img_shape = [None, X[0].shape[0],  X[0].shape[0], 3]
    imgs = tf.placeholder(tf.float32, img_shape)
    vgg = vgg16(imgs, 'vgg16_weights.npz', sess)
    for img in tqdm(X):
        img_for_processing = imresize(img, X[0].shape)
        prob = sess.run(vgg.probs, feed_dict={vgg.imgs: [img_for_processing]})[0]
        preds = (np.argsort(prob)[::-1])[0:5]
        result = list(map(lambda p: (class_names[p], prob[p]), preds))
        results += result
    sys.stdout = old_stdout
    return results

In [5]:
def predict_user(username):
    photos = []
    for img in glob.glob(username + '/*.jpg'):
        photo = imread(img, mode='RGB')
        photo = imresize(photo, (224, 224))
#         photo = imresize(photo, (photo.shape[0], photo.shape[1]))
        photos.append(photo)
    return predict(photos)
        

In [6]:
# results = predict_user('alexm.shots')

100%|██████████| 18/18 [00:08<00:00,  1.83it/s]


In [32]:
def process_result(results):
    classes = {}
    norm_const = 0
    for obj in results:
        norm_const += obj[1]
        if obj[0] in classes:
            classes[obj[0]] += obj[1]
        else:
            classes[obj[0]] = obj[1]
    return list(map(lambda (k,v): (k, v / norm_const), classes.iteritems()))
            

In [16]:
# results = process_result(results)

TypeError: unsupported operand type(s) for +=: 'int' and 'str'

In [30]:
# eps = 1e-6
# assert(np.sum(results.values()) > 1-eps and np.sum(results.values()) < 1 + eps)

In [31]:
# results

{'abaya': 0.0097773387932886605,
 'ambulance': 0.0063973960735215418,
 'analog clock': 0.00072038644824019295,
 'bannister, banister, balustrade, balusters, handrail': 0.0055981315652788421,
 'beacon, lighthouse, beacon light, pharos': 0.0066649676893195458,
 'bell cote, bell cot': 0.12830138069211849,
 'breakwater, groin, groyne, mole, bulwark, seawall, jetty': 0.12863491690480205,
 'cab, hack, taxi, taxicab': 0.032022967893453977,
 'car mirror': 0.0027821518576026877,
 'castle': 0.0043781140094456203,
 'church, church building': 0.012727157015236844,
 'cinema, movie theater, movie theatre, movie house, picture palace': 0.0060034847105358566,
 'computer keyboard, keypad': 0.011490867330872586,
 'dome': 0.015764836118259955,
 'grasshopper, hopper': 0.0021861216775396412,
 'greenhouse, nursery, glasshouse': 0.0059842243877584632,
 'guillotine': 0.02601590172059489,
 'harp': 0.0032184806025635816,
 'lakeside, lakeshore': 0.087597539466945998,
 'lampshade, lamp shade': 0.00323532318116853